# Multi timeframe momentum
Cílen je nalézt kombinaci trendu, kdy je velmi pravdě+podobné, že trend bude o určitou hodnootu ještě pokračovat.

jsou počítány linregression úhly pro více timeframů a délku oken

Pro každou kombinaci je daný parametr nad kterým musí být. Pokud je nad všemi pak je entry (short/long).

Zvážit i nějaký kumulativní počítadlo anglů - něco jako trend kummulátor.

In [ ]:
from v2realbot.tools.loadbatch import load_batch
from v2realbot.utils.utils import zoneNY
import pandas as pd
import numpy as np
import vectorbtpro as vbt
from itables import init_notebook_mode, show
import datetime
from itertools import product
from v2realbot.config import ACCOUNT1_PAPER_API_KEY, ACCOUNT1_PAPER_SECRET_KEY, DATA_DIR

init_notebook_mode(all_interactive=True)

vbt.settings.set_theme("dark")
vbt.settings['plotting']['layout']['width'] = 1280
vbt.settings.plotting.auto_rangebreaks = True
# Set the option to display with pagination
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_rows', 10)  # Number of rows per page

# Define the market open and close times
market_open = datetime.time(9, 30)
market_close = datetime.time(16, 0)
entry_window_opens = 1
entry_window_closes = 370

forced_exit_start = 380
forced_exit_end = 390

#LOAD FROM BATCH
# res, df = load_batch(batch_id="f1ac6651", #138170bc 0fb5043a  bde6d0be f1ac6651
#                      space_resolution_evenly=False,
#                      indicators_columns=["Rsi14"],
#                      main_session_only=True,
#                      verbose = False)
# if res < 0:
#     print("Error" + str(res) + str(df))
# df = df["bars"]

# basic_data = vbt.Data.from_data(vbt.symbol_dict({"BAC": df}), tz_convert=zoneNY)
# #m1_data = basic_data[['Open', 'High', 'Low', 'Close', 'Volume']]
# basic_data = basic_data.transform(lambda df: df.between_time('09:30', '16:00'))
# #basic_data.info()

#LOAD FROM PARQUET
#list all files is dir directory with parquet extension
dir = DATA_DIR + "/notebooks/"
import os
files = [f for f in os.listdir(dir) if f.endswith(".parquet")]
print('\n'.join(map(str, files)))
file_name = "ohlcv_df-BAC-2023-01-01T09_30_00-2024-05-25T15_30_00-47BCFOPUVWZ-100.parquet"
ohlcv_df = pd.read_parquet(dir+file_name,engine='pyarrow')
#filter ohlcv_df to certain date range (assuming datetime index)
ohlcv_df = ohlcv_df.loc["2024-02-12 10:30":"2024-02-14 12:00"]

#add vwap column to ohlcv_df
#ohlcv_df["hlcc4"] = (ohlcv_df["close"] + ohlcv_df["high"] + ohlcv_df["low"] + ohlcv_df["close"]) / 4

basic_data = vbt.Data.from_data(vbt.symbol_dict({"BAC": ohlcv_df}), tz_convert=zoneNY)
ohlcv_df= None

## Add resample function to custom columns

In [ ]:
from vectorbtpro.utils.config import merge_dicts, Config, HybridConfig
from vectorbtpro import _typing as tp
from vectorbtpro.generic import nb as generic_nb

_feature_config: tp.ClassVar[Config] = HybridConfig(
    {
        "buyvolume": dict(
            resample_func=lambda self, obj, resampler: obj.vbt.resample_apply(
                resampler,
                generic_nb.sum_reduce_nb,
            )
        ),
        "sellvolume": dict(
            resample_func=lambda self, obj, resampler: obj.vbt.resample_apply(
                resampler,
                generic_nb.sum_reduce_nb,
            )
        )
    }
)

basic_data._feature_config = _feature_config

In [ ]:
#asic_data.stats()
basic_data.wrapper.index.normalize().nunique()

In [ ]:
t1data = basic_data[['open', 'high', 'low', 'close', 'volume','vwap','buyvolume','sellvolume']].resample("1T")

In [ ]:
t1data.data["BAC"].buyvolume

In [ ]:
t1data.data["BAC"].sellvolume

In [ ]:
buyvolume = t1data.data["BAC"].buyvolume
sellvolume = t1data.data["BAC"].sellvolume
totalvolume = buyvolume + sellvolume

#adjust to minimal value to avoid division by zero
sellvolume_adjusted = sellvolume.replace(0, 1e-10)
oibratio = buyvolume / sellvolume

#cumulative order flow (net difference)
cof = buyvolume - sellvolume

# Calculate the order imbalance (net differene) normalize the order imbalance by calculating the difference between buy and sell volumes and then scaling it by the total volume.
order_imbalance = cof / totalvolume

order_imbalance_allvolume = cof / t1data.data["BAC"].volume

In [ ]:
order_imbalance

In [ ]:
#priminds list (cena), secinds list (napr. rsi), close, voluminds (volume based)
def plot_2y_close(priminds, secinds, close, volume):
    fig = vbt.make_subplots(rows=2, cols=1, shared_xaxes=True, 
                            specs=[[{"secondary_y": True}], [{"secondary_y": False}]], 
                            vertical_spacing=0.02, subplot_titles=("Price and Indicators", "Volume"))

    # Plotting the close price
    close.vbt.plot(fig=fig, add_trace_kwargs=dict(secondary_y=False,row=1, col=1), trace_kwargs=dict(line=dict(color="blue")))
    
    # Plotting primary indicators on the first row
    for ind in priminds:
        if isinstance(ind, pd.Series):
            ind = ind.vbt
        ind.plot(fig=fig, add_trace_kwargs=dict(secondary_y=False, row=1, col=1))
    
    # Plotting secondary indicators on the first row
    for ind in secinds:
        #ind = ind.rename(str(ind.name))
        if isinstance(ind, pd.Series):
            ind = ind.vbt
        ind.plot(fig=fig, add_trace_kwargs=dict(secondary_y=True, row=1, col=1))
    
    for indvolume in volume:
        # Plotting the volume on the second row
        indvolume.rename(str(indvolume.name)).vbt.barplot(fig=fig, add_trace_kwargs=dict(secondary_y=False, row=2, col=1))
        #vbt.Bar(indvolume, fig=fig, add_trace_kwargs=dict(secondary_y=False, row=2, col=1))
    
    return fig

plot_2y_close([], [cof,oibratio], t1data.close, [t1data.data["BAC"].buyvolume, t1data.data["BAC"].sellvolume, t1data.volume])


In [ ]:
%matplotlib inline
t0data = basic_data
t1data = basic_data[['open', 'high', 'low', 'close', 'volume','vwap']].resample("1T")
t2data = basic_data[['open', 'high', 'low', 'close', 'volume','vwap']].resample("15T")
t3data = basic_data[['open', 'high', 'low', 'close', 'volume','vwap']].resample("30T")
t4data = basic_data[['open', 'high', 'low', 'close', 'volume', 'vwap']].resample("D").dropna()

t1data = t1data.transform(lambda df: df.between_time('09:30', '16:00').dropna())
t2data = t2data.transform(lambda df: df.between_time('09:30', '16:00').dropna())
t3data = t3data.transform(lambda df: df.between_time('09:30', '16:00').dropna())

#30min data to daily
# t4data = t3data.resample("D").dropna()

#t4data = t4data.transform(lambda df: df.between_time('09:30', '16:00').dropna())
#m1data.data["SPY"].info()

#m1data.data["SPY"].vbt.ohlcv.plot()
#h2data.data["SPY"].vbt.ohlcv.plot()
#ddata.data["SPY"]
t2data.data["BAC"].vbt.ohlcv.plot().show()


#t4data.data["BAC"]

In [ ]:
t2data.close

#in df remove rows with nan


In [ ]:
#realign na 1T = t1data + oriznout main session
t2data_vwap = t2data.vwap.vbt.realign_closing("1T").between_time('09:30', '16:00').dropna()
t3data_vwap = t3data.vwap.vbt.realign_closing("1T").between_time('09:30', '16:00').dropna()
t4data_vwap = t4data.vwap.vbt.realign_closing("1T").dropna()

In [ ]:
t2data_vwap

In [ ]:
def plot_2y_close(priminds, secinds, close):
    fig = vbt.make_subplots(rows=1, cols=1, shared_xaxes=True, specs=[[{"secondary_y": True}]], vertical_spacing=0.02, subplot_titles=("MOM", "Price" ))
    close.vbt.plot(fig=fig, add_trace_kwargs=dict(secondary_y=False), trace_kwargs=dict(line=dict(color="blue")))
    for ind in priminds:
        if isinstance(ind, pd.Series):
            ind = ind.vbt
        ind.plot(fig=fig, add_trace_kwargs=dict(secondary_y=False))
    for ind in secinds:
        if isinstance(ind, pd.Series):
            ind = ind.vbt
        ind.plot(fig=fig, add_trace_kwargs=dict(secondary_y=True))
    return fig

In [ ]:
t4data.clos.vbt 



In [ ]:
obvind = vbt.indicator.obv.run(t1data.close, t1data.volume)

In [ ]:
t1_lengtgh = 15
t2_length = 15
t3_length = 15
t4_length = 5
t1_th = 0.1
t2_th = 0.1
t3_th = 0.1
t4_th = 0.1



#minute
t1slope = vbt.indicator("talib:LINEARREG_SLOPE ").run(t1data.close, timeperiod=t1_lengtgh) # -0.09, 0.09
t2slope = vbt.indicator("talib:LINEARREG_SLOPE ").run(t2data.vwap, timeperiod=t2_length) #   -0.08 , 0.079
t3slope = vbt.indicator("talib:LINEARREG_SLOPE ").run(t3data.vwap, timeperiod=t3_length) #   -0.08, 0.08
#daily
t4slope = vbt.indicator("talib:LINEARREG_SLOPE ").run(t4data.vwap, timeperiod=t4_length) #   -0.1, 0.09

plot_2y_close(priminds=[], secinds=[t1slope, t2slope, t3slope, t4slope], close=t1data.close).show()


In [ ]:
#thirtymin_slope = thirtymin_slope.real.rename("30min") #timto se prejmenuje real na 30min
t3slope = t3slope.real.vbt.realign_closing("1T").between_time('09:30', '16:00').dropna()
##filter daily_slope_to_compare to only monday to friday
t3slope = t3slope[t3slope.index.dayofweek < 5]

#t3slope.info()

t2slope = t2slope.real.vbt.realign_closing("1T").between_time('09:30', '16:00').dropna()
##filter daily_slope_to_compare to only monday to friday
t2slope = t2slope[t2slope.index.dayofweek < 5]

t2slope.info()

In [ ]:
oibratio

In [ ]:

#
short_entries = order_imbalance.vbt < 0.0002
#short_entries = oibratio.vbt < 0.01
short_entries.value_counts()

entries = order_imbalance.vbt > 0.7
#entries = oibratio.vbt > 10
entries.value_counts()

In [ ]:
fig = vbt.make_subplots(rows=3, cols=1, shared_xaxes=True, 
                        specs=[[{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": False}]], 
                        vertical_spacing=0.02, subplot_titles=("Price and Indicators", "Volume"))
t1data.data["BAC"].vbt.ohlcv.plot(fig=fig, add_trace_kwargs=dict(secondary_y=False, row=1, col=1))
#oibratio.vbt.plot(fig=fig, add_trace_kwargs=dict(secondary_y=True, row=1, col=1))
order_imbalance.vbt.plot(fig=fig, add_trace_kwargs=dict(secondary_y=True, row=1, col=1))
entries.vbt.signals.plot_as_entries(t1data.close, fig=fig, trace_kwargs=dict(name="LONGS",
                                                                                    line=dict(color="#ffe476"),
                                                                                    marker=dict(color="limegreen"),
                                                                                    fill=None,
                                                                                    connectgaps=True,
                                                                                    ), add_trace_kwargs=dict(secondary_y=False, row=1, col=1))

short_entries.vbt.signals.plot_as_entries(t1data.close, fig=fig, trace_kwargs=dict(name="SHORTS",
                                                                                    line=dict(color="#ffe476"),
                                                                                    marker=dict(color="red"),
                                                                                    fill=None,
                                                                                    connectgaps=True,
                                                                                    ), add_trace_kwargs=dict(secondary_y=False, row=1, col=1))


In [ ]:
# thirtymin_slope_to_compare.vbt.xloc["04-16-2024"].get()
thirty_down_signal.vbt.xloc["04-16-2024"].get()


In [ ]:
#short_signal = t1slope.real_below(t1_th) & t2slope.real_below(t2_th) & t3slope.real_below(t3_th) & t4slope.real_below(t4_th)
#long_signal = t1slope.real_above(t1_th) & t2slope.real_above(t2_th) & t3slope.real_above(t3_th) & t4slope.real_above(t4_th)

#test na daily s reversem  crossed 0
short_signal = t2slope.vbt < -0.01 & t3slope.vbt < -0.01  #min value of threshold
long_signal = t2slope.vbt > 0.01 & t3slope.vbt > 0.01  #min

# thirty_up_signal = t3slope.vbt.crossed_above(0.01)
# thirty_down_signal = t3slope.vbt.crossed_below(-0.01)

fig = plot_2y_close(priminds=[], secinds=[t3slope], close=t1data.close)
#short_signal.vbt.signals.plot_as_entries(basic_data.close, fig=fig)

short_signal.vbt.signals.plot_as_entries(t1data.close, fig=fig, trace_kwargs=dict(name="SHORTS",
                                                                                    line=dict(color="#ffe476"),
                                                                                    marker=dict(color="red", symbol="triangle-down"),
                                                                                    fill=None,
                                                                                    connectgaps=True,
                                                                                    ))
long_signal.vbt.signals.plot_as_entries(t1data.close, fig=fig, trace_kwargs=dict(name="LONGS",
                                                                                    line=dict(color="#ffe476"),
                                                                                    marker=dict(color="limegreen"),
                                                                                    fill=None,
                                                                                    connectgaps=True,
                                                                                    ))

# thirty_down_signal.vbt.signals.plot_as_entries(t1data.close, fig=fig, trace_kwargs=dict(name="DOWN30",
#                                                                                     line=dict(color="#ffe476"),
#                                                                                     marker=dict(color="yellow", symbol="triangle-down"),
#                                                                                     fill=None,
#                                                                                     connectgaps=True,
#                                                                                     ))
# thirty_up_signal.vbt.signals.plot_as_entries(t1data.close, fig=fig, trace_kwargs=dict(name="UP30",
#                                                                                     line=dict(color="#ffe476"),
#                                                                                     marker=dict(color="grey"),
#                                                                                     fill=None,
#                                                                                     connectgaps=True,
#                                                                                     ))

# thirtymin_slope_to_compare.vbt.plot(fig=fig, add_trace_kwargs=dict(secondary_y=True), trace_kwargs=dict(name="30min slope",
#                                                                                         line=dict(color="yellow"),                                                                                     
#                                                                                         fill=None,
#                                                                                         connectgaps=True,
#                                                                                         ))

fig.show()
# print("short signal")
# print(short_signal.value_counts())

#forced_exit = pd.Series(False, index=close.index)
forced_exit = basic_data.symbol_wrapper.fill(False)
#entry_window_open = pd.Series(False, index=close.index)
entry_window_open=  basic_data.symbol_wrapper.fill(False)

# Calculate the time difference in minutes from market open for each timestamp
elapsed_min_from_open = (forced_exit.index.hour - market_open.hour) * 60 + (forced_exit.index.minute - market_open.minute)

entry_window_open[(elapsed_min_from_open >= entry_window_opens) & (elapsed_min_from_open < entry_window_closes)] = True

#print(entry_window_open.value_counts())

forced_exit[(elapsed_min_from_open >= forced_exit_start) & (elapsed_min_from_open < forced_exit_end)] = True
short_entries = (short_signal & entry_window_open)
short_exits = forced_exit

entries = (long_signal & entry_window_open)
exits = forced_exit
#long_entries.info()
#number of trues and falses in long_entries
# print(short_exits.value_counts())
# print(short_entries.value_counts())

#fig = plot_2y_close([],[momshort, rocp], close)
#short_signal.vbt.signals.plot_as_entries(close, fig=fig, add_trace_kwargs=dict(secondary_y=False))
#print(sl_stop)
#short_entries=short_entries, short_exits=short_exits,
# pf = vbt.Portfolio.from_signals(close=basic_data, entries=short_entries, exits=exits, tsl_stop=0.005, tp_stop = 0.05, fees=0.0167/100, freq="1s") #sl_stop=sl_stop, tp_stop = sl_stop,

# pf.stats()


In [ ]:
# pf.plot()

In [ ]:
pf.get_drawdowns().records_readable

In [ ]:
pf.orders.records_readable